# Gerador de Perguntas Dinâmico Utilizando T5 e PTT5 em Português/Inglês 
## Parte 1 - Preparação dos Dados

*Baseado no incrível trabalho realizado por Suraj Patil* -  [Question Generation 
Using Transformers](https://github.com/patil-suraj/question_generation)  

--- 

1.   A carga dos datasets foi implementada utilizando a biblioteca  [nlp.Dataset da Huggingface](https://huggingface.co/docs/datasets/v0.3.0/index.html)
2.   Os datasets para treinos foram baseados no dataset [SQUAD 1.1](https://rajpurkar.github.io/SQuAD-explorer/) e [SQUAD 1.1 pt_br](https://drive.google.com/file/d/1Q0IaIlv2h2BC468MwUFmUST0EyN7gNkn/view?usp=sharing) (publicado e revisado pelo grupo Deep Learning Brasil [clique aqui para mais detalhes](https://medium.com/@pierre_guillou/nlp-modelo-de-question-answering-em-qualquer-idioma-baseado-no-bert-base-estudo-de-caso-em-12093d385e78))
3.    Seguindo a arquitetura recomendada pela Huggingface utilizou-se classes que tratam o download do dataset: Em inglês é realizado o download (adaptada a classe criada por *Suraj Patil*), já em português utilizo uma classe que utiliza arquivos locais (funciona no google drive).
4.    Após o tratamento da carga as bases de treino e validação precisam ser preparadas para entrada em um modelo T5 (Text To Text Transfer Transformer).
5.    A preparação do dataset representa o tratamento do texto para entrada em um modelo T5, gerando os embeddings, attetion masks e também os targets que no nosso modelo representam as perguntas geradas. 
6.    No final os datasets tratados são armazenados para posterior uso, indicando no respectivo nome o modelo para o qual deve ser utilizado.
7.    Um novo tokenizer será criado para o modelo já que precisamos adicionar novos tokens (special_tokens).






In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install nlp
!pip install sentencepiece
!pip install datasets
!pip install jsonlines

In [ ]:
import os
import logging
from dataclasses import dataclass, field
from typing import Dict, List, Optional

import json
import jsonlines
from tqdm import tqdm

import torch
import nlp
from transformers import T5Tokenizer, BartTokenizer, HfArgumentParser

import datasets
from datasets import load_dataset, Dataset, set_caching_enabled


In [ ]:
GDRIVE_PATH = '/content/drive/MyDrive'

LANGUAGE_PORTUGUES = "pt_br"
LANGUAGE_ENGLISH = "en"

# Dataset em ingles é baixado por URL, não necessita de caminho
DATASET_SQUAD_PT_BR_PATH = os.path.join(GDRIVE_PATH, 'dataset', 'SQuAD-Portugues')

TRAIN_PT_FILE_NAME= os.path.join(DATASET_SQUAD_PT_BR_PATH, 'squad-train-v1.1.json')
VALIDATION_PT_FILE_NAME= os.path.join(DATASET_SQUAD_PT_BR_PATH, 'squad-dev-v1.1.json')

DATASET_LOADER_CLASSPATH_PORTUGUES = os.path.join(GDRIVE_PATH, 'python/question-generator/squad_pt_br/')
DATASET_LOADER_CLASSPATH_ENGLISH = os.path.join(GDRIVE_PATH, 'python/question-generator/squad_multitask/')

FINAL_OUTPUT_TRAIN_FILE_NAME= os.path.join(DATASET_SQUAD_PT_BR_PATH, 'train_data_qg_pt_br_t5.pt')
FINAL_OUTPUT_DEV_FILE_NAME= os.path.join(DATASET_SQUAD_PT_BR_PATH, 'valid_data_e2e_qg_pt_br_t5.pt')

DATASET_PREPARED_OUTPUT_PATH = os.path.join(GDRIVE_PATH, 'dataset', 'question-generator')
DATASET_PREPARED_PREFIX = 'data_qg_'
DATASET_PREPARED_SUFFIX = '.pt'

MODEL_PTT5_BASE = 'unicamp-dl/ptt5-base-portuguese-vocab'
MODEL_PTT5_LARGE = 'unicamp-dl/ptt5-large-portuguese-vocab'
MODEL_T5_BASE= "t5-base"

#TOKENIZER_PATH = os.path.join(GDRIVE_PATH, 'model', 'tokenizer', 'ptt5-base-portuguese-vocab-qg-tokenizer')
TOKENIZER_PATH = os.path.join(GDRIVE_PATH, 'model', 'tokenizer')

MAX_SOURCE_LENGTH=512
MAX_TARGET_LENGTH=32

In [ ]:
def get_gpu_info():
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
        print('and then re-execute this cell.')
    else:
        print(gpu_info)

In [ ]:
logger = logging.getLogger(__name__)
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO
    )


#### Classe para preparação de um Dataset no formato esperado por um modelo T5 para processamento. 
- Substitui as marcações geradas na carga do dataset por tokens especiais que foram adicionados no tokenizador.
- Geração das colunas: source_ids, attention_masks, target_ids. Target_ids representam os embeddings da perguntas geradas, separadas pelo token de separação.
- Essa classe não carrega do Dataset, o carregamento teve a responsabilidade delegada para classes especificas, uma para português e outra para inglês.

In [ ]:
class DataProcessor:
    def __init__(self, tokenizer, max_source_length=512, max_target_length=32):
        self.tokenizer = tokenizer
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length
        
        self.hl_token = "<hl>"
        self.sep_token = "<sep>"
        
    def process(self, dataset):
        dataset = dataset.map(self._add_eos_examples)
        dataset = dataset.map(self._add_special_tokens)
        dataset = dataset.map(self._convert_to_features, batched=True)
        
        return dataset
  
    def _add_eos_examples(self, example):
        example['source_text'] = example['source_text'] + " </s>"
        example['target_text'] = example['target_text'] + " </s>"
        return example
  
    def _add_special_tokens(self, example):
        example['source_text'] = example['source_text'].replace("{hl_token}", self.hl_token)    
        example['target_text'] = example['target_text'].replace("{sep_token}", self.sep_token)
        return example
  
    # tokenize the examples
    def _convert_to_features(self, example_batch):
        source_encoding = self.tokenizer.batch_encode_plus(
            example_batch['source_text'],
            max_length=self.max_source_length,
            padding='max_length',
            pad_to_max_length=True,
            truncation=True, 
        )
        target_encoding = self.tokenizer.batch_encode_plus(
            example_batch['target_text'],
            max_length=self.max_target_length,
            padding='max_length',
            pad_to_max_length=True,
            truncation=True, 
        )

        encodings = {
            'source_ids': source_encoding['input_ids'], 
            'target_ids': target_encoding['input_ids'],
            'attention_mask': source_encoding['attention_mask'],
        }

        return encodings


#### Carga dos Datasets
#####squad_pt_br e squad_multitask representam o caminho das classes criadas para carga do dataset. No momento da carga o contexto e as perguntas do SQUAD são tratadas para incorporar o comando para o qual o modelo T5 será treinado posteriormente: "generate questions:" em inglês e "gerar perguntas:" em português.
Exemplo: (... para diminuir o tamanho do texto)
```
{
            "title": "Super_Bowl_50",
            "paragraphs": [{
                    "context": "O Super Bowl 50 foi um jogo de futebol americano para determinar o campeão da Liga Nacional de Futebol Americano (NFL) para a temporada de 2015. O campeão da Conferência Americana de Futebol Americano (AFC), Denver Broncos, derrotou o campeão da Conferência Nacional de Futebol Americano (NFC), Carolina Panthers 24, para ganhar seu terceiro título do Super Bowl. O jogo foi disputado em 7 de fevereiro de 2016, no Levi's Stadium na área da Baía de São Francisco em Santa Clara, Califórnia. Como este foi o 50o Super Bowl, a liga enfatizou o "aniversário de ouro" com várias iniciativas com temas de ouro, além de suspender temporariamente a tradição de nomear cada jogo do Super Bowl com números romanos (sob os quais o jogo seria conhecido como " Super Bowl L "), de modo que o logotipo possa exibir com destaque os algarismos arábicos 50.",
                    "qas": [{  
                            ...                          
                            "question": "Qual time da NFL representou a AFC no Super Bowl 50?",
                            ...
                             }, 
                            {
                            ...
                            "question": "Onde aconteceu o Super Bowl 50?"
                            ...
                            },
                            ...
                            "question": "Em que ano o Denver Broncos conquistou o título do Super Bowl pela terceira vez?"
                            ...
                            },                            
						...
                    ]
                }
		...
		]
}
```

**vai ser transformado em**

```
{
    'source_text': 'gerar perguntas: O Super Bowl 50 foi um jogo de futebol americano para determinar o campeão da Liga Nacional de Futebol Americano (NFL) para a temporada de 2015. O campeão da Conferência Americana de Futebol Americano (AFC), Denver Broncos, derrotou o campeão da Conferência Nacional de Futebol Americano (NFC), Carolina Panthers 24, para ganhar seu terceiro título do Super Bowl. O jogo foi disputado em 7 de fevereiro de 2016, no Levi's Stadium na área da Baía de São Francisco em Santa Clara, Califórnia. Como este foi o 50o Super Bowl, a liga enfatizou o "aniversário de ouro" com várias iniciativas com temas de ouro, além de suspender temporariamente a tradição de nomear cada jogo do Super Bowl com números romanos (sob os quais o jogo seria conhecido como " Super Bowl L "), de modo que o logotipo possa exibir com destaque os algarismos ar\u00e1bicos 50.'
    'target_text': 'Qual time da NFL representou a AFC no Super Bowl 50? {sep_token} Onde aconteceu o Super Bowl 50? {sep_token} Em que ano o Denver Broncos conquistou o título do Super Bowl pela terceira vez?'
}
```




In [ ]:
def carrega_datasets(language=LANGUAGE_PORTUGUES):
    if language == LANGUAGE_PORTUGUES:
        train_dataset:Dataset = nlp.load_dataset(DATASET_LOADER_CLASSPATH_PORTUGUES, data_dir=DATASET_SQUAD_PT_BR_PATH,split=nlp.Split.TRAIN)
        validation_dataset:Dataset = nlp.load_dataset(DATASET_LOADER_CLASSPATH_PORTUGUES, data_dir=DATASET_SQUAD_PT_BR_PATH,split=nlp.Split.VALIDATION)
    else: 
        train_dataset:Dataset = nlp.load_dataset(DATASET_LOADER_CLASSPATH_ENGLISH,split=nlp.Split.TRAIN)
        validation_dataset:Dataset = nlp.load_dataset(DATASET_LOADER_CLASSPATH_ENGLISH,split=nlp.Split.VALIDATION)

    print('Tamanho train_dataset', len(train_dataset))
    print('Tamanho validations_dataset', len(validation_dataset))
    return train_dataset, validation_dataset
 

In [ ]:
def carrega_tokenizer(model_name):
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    tokenizer.add_tokens(['<sep>', '<hl>'])

    return tokenizer

In [ ]:
def processa_dataset(tokenizer: T5Tokenizer, train_dataset: Dataset, validation_dataset: Dataset):
    processor = DataProcessor(
            tokenizer,
            max_source_length=MAX_SOURCE_LENGTH,
            max_target_length=MAX_TARGET_LENGTH
        )
    train_dataset = processor.process(train_dataset)
    validation_dataset = processor.process(validation_dataset)

    return train_dataset, validation_dataset

In [ ]:
def save_dataset(train_dataset: Dataset, validation_dataset: Dataset, train_file_name: str, validation_file_name: str, tokenizer_path: str):
    columns = ["source_ids", "target_ids", "attention_mask"]
    train_dataset.set_format(type='torch', columns=columns)
    validation_dataset.set_format(type='torch', columns=columns)

    torch.save(train_dataset, train_file_name)
    logger.info(f"saved train dataset at {train_file_name}")

    torch.save(validation_dataset, validation_file_name)
    logger.info(f"saved validation dataset at {validation_file_name}")

    if not os.path.exists(tokenizer_path):
        os.mkdir(tokenizer_path)
    tokenizer.save_pretrained(tokenizer_path)
    logger.info(f"saved tokenizer at {tokenizer_path}")

##### Execução
- Tratamento do modelo que será utilizado e respectivos paths
- Definição do idioma para correto carregamento do dataset.
- Preparação do dataset.
- Salvar o dataset com nome relacionado ao modelo.
- Nesse primeiro passo somente o tokenizer é utilizado.

In [ ]:
model_path = MODEL_T5_BASE #MODEL_PTT5_LARGE #MODEL_PTT5_BASE #
model_name = model_path.split(sep="/")[-1]
tokenizer_path = os.path.join(TOKENIZER_PATH, model_name + '-qg-tokenizer')
dataset_train_prepared_output_filename = os.path.join(DATASET_PREPARED_OUTPUT_PATH, 
                                                      'train_'+ DATASET_PREPARED_PREFIX + model_name + DATASET_PREPARED_SUFFIX)
dataset_validation_prepared_output_filename = os.path.join(DATASET_PREPARED_OUTPUT_PATH, 
                                                           'valid_'+ DATASET_PREPARED_PREFIX + model_name + DATASET_PREPARED_SUFFIX)
                                            
tokenizer = carrega_tokenizer(model_name=model_path)
#train_dataset, validation_dataset = carrega_datasets(language=LANGUAGE_PORTUGUES)
train_dataset, validation_dataset = carrega_datasets(language=LANGUAGE_ENGLISH)
train_dataset, validation_dataset = processa_dataset(tokenizer=tokenizer,
                                                     train_dataset=train_dataset, 
                                                     validation_dataset=validation_dataset)

save_dataset(train_dataset=train_dataset,
             validation_dataset=validation_dataset,
             train_file_name=dataset_train_prepared_output_filename,
             validation_file_name=dataset_validation_prepared_output_filename,
             tokenizer_path=tokenizer_path)

07/10/2021 08:02:00 - INFO - nlp.load -   Checking /content/drive/MyDrive/python/question-generator/squad_multitask/squad_multitask.py for additional imports.
07/10/2021 08:02:00 - INFO - filelock -   Lock 140279484469584 acquired on /content/drive/MyDrive/python/question-generator/squad_multitask/squad_multitask.py.lock
07/10/2021 08:02:00 - INFO - nlp.load -   Found main folder for dataset /content/drive/MyDrive/python/question-generator/squad_multitask/squad_multitask.py at /usr/local/lib/python3.7/dist-packages/nlp/datasets/squad_multitask
07/10/2021 08:02:00 - INFO - nlp.load -   Found specific version folder for dataset /content/drive/MyDrive/python/question-generator/squad_multitask/squad_multitask.py at /usr/local/lib/python3.7/dist-packages/nlp/datasets/squad_multitask/a429d5db5cd26b040175b11e124c1fb4e4192a36231ef5b46a43c5b067cb24af
07/10/2021 08:02:00 - INFO - nlp.load -   Found script file from /content/drive/MyDrive/python/question-generator/squad_multitask/squad_multitask.

Tamanho train_dataset 18896
Tamanho validations_dataset 2067


07/10/2021 08:02:01 - INFO - nlp.arrow_writer -   Done writing 18896 examples in 21149487 bytes /root/.cache/huggingface/datasets/squad_multitask/highlight_qg_format/1.0.0/a429d5db5cd26b040175b11e124c1fb4e4192a36231ef5b46a43c5b067cb24af/tmp8e9dt5mq.
07/10/2021 08:02:01 - INFO - nlp.arrow_dataset -   Caching processed dataset at /root/.cache/huggingface/datasets/squad_multitask/highlight_qg_format/1.0.0/a429d5db5cd26b040175b11e124c1fb4e4192a36231ef5b46a43c5b067cb24af/cache-7f0403407558aedbc38808fde1d9b12b.arrow


07/10/2021 08:02:02 - INFO - nlp.arrow_writer -   Done writing 18896 examples in 20623893 bytes /root/.cache/huggingface/datasets/squad_multitask/highlight_qg_format/1.0.0/a429d5db5cd26b040175b11e124c1fb4e4192a36231ef5b46a43c5b067cb24af/tmpbjc2q65f.
/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:191: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
07/10/2021 08:02:02 - INFO - nlp.arrow_dataset -   Caching processed dataset at /root/.cache/huggingface/datasets/squad_multitask/highlight_qg_format/1.0.0/a429d5db5cd26b040175b11e124c1fb4e4192a36231ef5b46a43c5b067cb24af/cache-0f178ce77edbf3172f8515e25be96d3b.arrow


07/10/2021 08:02:48 - INFO - nlp.arrow_writer -   Done writing 18896 examples in 180484281 bytes /root/.cache/huggingface/datasets/squad_multitask/highlight_qg_format/1.0.0/a429d5db5cd26b040175b11e124c1fb4e4192a36231ef5b46a43c5b067cb24af/tmpcbhhs7fh.
07/10/2021 08:02:48 - INFO - nlp.arrow_dataset -   Caching processed dataset at /root/.cache/huggingface/datasets/squad_multitask/highlight_qg_format/1.0.0/a429d5db5cd26b040175b11e124c1fb4e4192a36231ef5b46a43c5b067cb24af/cache-4f279ddf3d91113eaf7d91e9a95d1697.arrow


07/10/2021 08:02:48 - INFO - nlp.arrow_writer -   Done writing 2067 examples in 2471901 bytes /root/.cache/huggingface/datasets/squad_multitask/highlight_qg_format/1.0.0/a429d5db5cd26b040175b11e124c1fb4e4192a36231ef5b46a43c5b067cb24af/tmpfbvgjm99.
07/10/2021 08:02:48 - INFO - nlp.arrow_dataset -   Caching processed dataset at /root/.cache/huggingface/datasets/squad_multitask/highlight_qg_format/1.0.0/a429d5db5cd26b040175b11e124c1fb4e4192a36231ef5b46a43c5b067cb24af/cache-ceab09788e10dca0b39add4d9858ca88.arrow


07/10/2021 08:02:48 - INFO - nlp.arrow_writer -   Done writing 2067 examples in 2408481 bytes /root/.cache/huggingface/datasets/squad_multitask/highlight_qg_format/1.0.0/a429d5db5cd26b040175b11e124c1fb4e4192a36231ef5b46a43c5b067cb24af/tmpn6mh3t89.
07/10/2021 08:02:48 - INFO - nlp.arrow_dataset -   Caching processed dataset at /root/.cache/huggingface/datasets/squad_multitask/highlight_qg_format/1.0.0/a429d5db5cd26b040175b11e124c1fb4e4192a36231ef5b46a43c5b067cb24af/cache-cfee7d7868f5861f704eff6b2f6fdb3e.arrow


07/10/2021 08:02:54 - INFO - nlp.arrow_writer -   Done writing 2067 examples in 19895337 bytes /root/.cache/huggingface/datasets/squad_multitask/highlight_qg_format/1.0.0/a429d5db5cd26b040175b11e124c1fb4e4192a36231ef5b46a43c5b067cb24af/tmpseb41mb_.
07/10/2021 08:02:54 - INFO - nlp.arrow_dataset -   Set __getitem__(key) output type to torch for ['source_ids', 'target_ids', 'attention_mask'] columns  (when key is int or slice) and don't output other (un-formated) columns.
07/10/2021 08:02:54 - INFO - nlp.arrow_dataset -   Set __getitem__(key) output type to torch for ['source_ids', 'target_ids', 'attention_mask'] columns  (when key is int or slice) and don't output other (un-formated) columns.


07/10/2021 08:02:55 - INFO - __main__ -   saved train dataset at /content/drive/MyDrive/dataset/question-generator/train_data_qgt5-base.pt
07/10/2021 08:02:56 - INFO - __main__ -   saved validation dataset at /content/drive/MyDrive/dataset/question-generator/valid_data_qgt5-base.pt
07/10/2021 08:02:56 - INFO - __main__ -   saved tokenizer at /content/drive/MyDrive/model/tokenizer/t5-base-qg-tokenizer
